In [39]:
# Домашнее задание № 2


1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

#### Подготовительный код

In [40]:
import pandas as pd


In [41]:
news = pd.read_csv("./materials.csv")
print(news.shape)
news.head(3)


(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [42]:
users = pd.read_csv("./users_articles.csv")
users.head(3)


,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


#### 1. Получаем векторные представления новостей

In [43]:
#!pip install gensim
#!pip install python-Levenshtein


In [44]:
from gensim.corpora.dictionary import Dictionary


In [45]:
import re
import numpy as np

from razdel import tokenize  # https://github.com/natasha/razdel
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

import pymorphy2


ImportError: cannot import name 'Sequence' from 'collections' (c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\collections\__init__.py)

In [ ]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()


AttributeError: module 'stopwords' has no attribute 'words'

In [ ]:
with open('./stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)


NameError: name 'stopword_ru' is not defined

In [ ]:
def clean_text(text):
    '''
    очистка текста

    на выходе очищеный текст

    '''
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub(
        "[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]

    # return " ".join(words)
    return text


cache = {}


def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)

    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-':  # [2]
            w = w[1:]
        if len(w) > 1:  # [3]
            if w in cache:  # [4]
                words_lem.append(cache[w])
            else:  # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)

    words_lem_without_stopwords = [
        i for i in words_lem if not i in stopword_ru]  # [6]

    return words_lem_without_stopwords


In [ ]:
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)


C:\Users\azhur\AppData\Local\Temp\ipykernel_13168\2296537027.py:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


In [ ]:
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)


NameError: name 'news' is not defined

In [ ]:
news['title']


NameError: name 'news' is not defined

In [ ]:
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]


NameError: name 'news' is not defined

In [ ]:
from gensim.models import LdaModel


In [ ]:
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25,
               id2word=common_dictionary)  # , passes=10)


NameError: name 'common_corpus' is not defined

In [ ]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)


NameError: name 'lda' is not defined

In [ ]:
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc]


NameError: name 'news' is not defined

In [ ]:
x = lda.show_topics(num_topics=25, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Below Code Prints Only Words
for topic, words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))


NameError: name 'lda' is not defined

In [ ]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(
        zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)


In [ ]:
topic_matrix = pd.DataFrame([get_lda_vector(text)
                            for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id'] +
                            ['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)


NameError: name 'pd' is not defined

#### Следующий шаг - векторные представления пользователей

In [ ]:
users.head(3)


NameError: name 'users' is not defined

In [ ]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[
                'topic_{}'.format(i) for i in range(25)]].values))


NameError: name 'topic_matrix' is not defined

In [ ]:
user_articles_list = users['articles'].iloc[33]


def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector


NameError: name 'users' is not defined

In [ ]:
get_user_embedding(user_articles_list)


NameError: name 'get_user_embedding' is not defined

In [ ]:
user_embeddings1 = pd.DataFrame(
    [i for i in users['articles'].apply(lambda x: get_user_embedding_median(x), 1)])
user_embeddings1.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings1['uid'] = users['uid'].values
user_embeddings1 = user_embeddings1[['uid'] +
                                    ['topic_{}'.format(i) for i in range(25)]]
user_embeddings1.head(3)


NameError: name 'pd' is not defined

In [ ]:
X1 = pd.merge(user_embeddings1, target, 'left')
X1.head(3)


NameError: name 'pd' is not defined

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1[['topic_{}'.format(i) for i in range(25)]],
                                                        X1['churn'], random_state=0)


NameError: name 'train_test_split' is not defined

In [ ]:
logreg1 = LogisticRegression()
# обучим
logreg1.fit(X_train1, y_train1)


In [ ]:
preds1 = logreg1.predict_proba(X_test1)[:, 1]
preds1[:10]


Перед расчетом метрик подготовим базу для таблицы сравнения метрик:

In [ ]:
metric_list = [[i]
               for i in [thresholds[ix], fscore[ix], precision[ix], recall[ix]]]
metric_list


In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test1, preds1)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix],
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))


In [ ]:
median_metrcs = [thresholds[ix], fscore[ix], precision[ix], recall[ix]]
for i in range(len(metric_list)):
    metric_list[i].append(median_metrcs[i])
metric_list


In [ ]:
font = {'size': 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(y_test1, preds1 > thresholds[ix])
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()


In [ ]:
roc_auc_score(y_test1, preds1)


#### Повторим п.2, но используя уже не медиану, а max

In [ ]:
def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector


In [ ]:
get_user_embedding_max(user_articles_list)


In [ ]:
user_embeddings2 = pd.DataFrame(
    [i for i in users['articles'].apply(lambda x: get_user_embedding_max(x), 1)])
user_embeddings2.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings2['uid'] = users['uid'].values
user_embeddings2 = user_embeddings2[['uid'] +
                                    ['topic_{}'.format(i) for i in range(25)]]
user_embeddings2.head(3)


In [ ]:
X2 = pd.merge(user_embeddings2, target, 'left')
X2.head(3)


In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2[['topic_{}'.format(i) for i in range(25)]],
                                                        X2['churn'], random_state=0)


In [ ]:
logreg2 = LogisticRegression()
# обучим
logreg2.fit(X_train2, y_train2)


In [ ]:
preds2 = logreg2.predict_proba(X_test2)[:, 1]
preds2[:10]


In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test2, preds2)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix],
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))


In [ ]:
median_metrcs = [thresholds[ix], fscore[ix], precision[ix], recall[ix]]
for i in range(len(metric_list)):
    metric_list[i].append(median_metrcs[i])
metric_list


In [ ]:
font = {'size': 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(y_test2, preds2 > thresholds[ix])
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()


In [ ]:
roc_auc_score(y_test2, preds2)

#### Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [ ]:
metric_df=pd.DataFrame(
    metric_list, columns=['mean','median','max'], index=['Treshhold','F-score','Precision', 'Recall']
).append(pd.DataFrame(
                    [[roc_auc_score(y_test, preds),roc_auc_score(y_test1, preds1),roc_auc_score(y_test2, preds2)]],
    columns=['mean','median','max'],
    index=['ROC-AUC']))
metric_df

Как видно из таблицы, наиболее эффективным оказался метод с использованием медианы для получения эмбеддинга пользователя, т.к. мы получаем более разреженный вектор заинтересованности пользователя, в следствии того, что что в отличии от среднего и максимального значения, медиана чаще будет равна нулю, на тех темах статей, оторые редко появляются в векторе статей читаемых пользователем. То есть интересы пользователя будут выражены наиболее явно.